# C. Elegans Data 

#### This notebook goes through an example from the talk of **Phil Kidd's** C. Elegans data, ultimately showing how `fastplotlib` can be used as a tool for examining behavioral data

In [ ]:
import fastplotlib as fpl
import numpy as np
import pickle
from ipywidgets import HBox, VBox

## Load the data

Dataset contains three worm tracks stored in a dictionary with keys "0", "1", and "2". Within each track data respective to each work including the raw images, positions, angular velocity etc.

In [ ]:
with open('demodata', 'rb') as f:
    data = pickle.load(f)
data.keys()

In [ ]:
worm_viewer = fpl.ImageWidget(data=[np.array(data[0]["images"]), np.array(data[1]["images"]), np.array(data[2]["images"])], cmap="gray", figure_kwargs={"size": (700, 560)})
worm_viewer.show()

## Let's look at an individual track

In [ ]:
track = data[1]

In [ ]:
# view the raw images 
worm_viewer = fpl.ImageWidget(data=np.array(track["images"]), cmap="gray", figure_kwargs={"size": (700, 560)})
worm_viewer.show()

## View corresponding behavior

In [ ]:
track_fig = fpl.Figure(names=[["Worm track"]],size=(900,700))

### Define a predetermined code for the behavioral ethogram

In [ ]:
# color code for ethogram
color_dict = {'forward run':'r','short turn':'b','forward curving':'g','omega':'k','pause':'y','reversal':'m','prereversal':'c','unclassified':'gray'}

### Define colors using `color_dict`

In [ ]:
# gives us a color for every datapoint, can then apply as a colormap to our line
colors = [color_dict[bhv] for bhv in track['behaviors2']]
colors[:10]

### Plot the xy position of the work over time

In [ ]:
track_graphic = track_fig[0, 0].add_line(np.column_stack([track['xsc'], track['ysc']]),colors=colors)

#### Add a point to track the position over time

In [ ]:
point = track_fig[0, 0].add_scatter(np.column_stack([track['xsc'][0], track['ysc'][0]]))
point.sizes = 10
point.colors = 'w'

### Plot the concentrations

In [ ]:
data_fig = fpl.Figure(names=[["Concentration plot"]], size=(600, 300))
data_graphic = data_fig[0, 0].add_line(np.column_stack([np.array(track['times']), np.array(track["xsc"])]),colors=colors)

In [ ]:
# add a linear selector
selector = data_graphic.add_linear_selector()

### Add an event handler to update the data

In [ ]:
def change_point(ev):
    global point
    global selector
    ix = ev['t']
    # update the xy position of the point 
    point.data = np.array([track['xsc'][ix], track['ysc'][ix], 0])
    # update the selection of the linear selector
    selector.selection = track['times'][ix]

In [ ]:
worm_viewer.add_event_handler(change_point)

## View the plots

In [ ]:
HBox([VBox([track_fig.show(), data_fig.show(maintain_aspect=False)]), worm_viewer.show()])